In [ ]:
!pip install ultralytics

In [ ]:
import shutil
import os

output_dir = '/content/outputt'
zip_file_name = '/content/outputt.zip'

# Create a zip archive of the output directory
shutil.make_archive(os.path.splitext(zip_file_name)[0], 'zip', output_dir)

print(f"The folder '{output_dir}' has been successfully zipped to '{zip_file_name}'")

The folder '/content/outputt' has been successfully zipped to '/content/outputt.zip'


In [ ]:
import os, glob, json
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

def ensure_dir(p):
    os.makedirs(p, exist_ok=True); return p

def save_instance_masks(res, mask_dir, basename):
    if res.masks is None: return 0, 0.0
    m = res.masks.data.cpu().numpy().astype(np.uint8)
    n = m.shape[0]
    combined = (m.sum(axis=0) > 0).astype(np.uint8) * 255
    cv2.imwrite(os.path.join(mask_dir, f"{basename}_mask.png"), combined)
    for i in range(n):
        inst = (m[i] > 0).astype(np.uint8) * 255
        cv2.imwrite(os.path.join(mask_dir, f"{basename}_mask_{i+1}.png"), inst)
    h, w = combined.shape
    area_ratio = combined.sum() / 255.0 / (h * w)
    return n, float(area_ratio)

def main():

    # ===========================
    #  HARD-CODED PATHS HERE
    # ===========================
    WEIGHTS_PATH = "/content/drive/MyDrive/Colab Notebooks/weights/best.pt"   # Your trained model
    SOURCE_PATH  = "/content/drive/MyDrive/Colab Notebooks/dataset/images/test"        # Folder with images
    outdir  = "outputt"                              # output folder
    imgsz   = 896
    conf    = 0.25
    # ===========================

    model = YOLO(WEIGHTS_PATH)

    image_paths = []
    for e in ("*.jpg","*.png","*.jpeg","*.bmp","*.tif","*.tiff"):
        image_paths += glob.glob(os.path.join(SOURCE_PATH, e))

    ann_dir = ensure_dir(os.path.join(outdir, "images_annotated"))
    json_dir = ensure_dir(os.path.join(outdir, "json"))
    mask_dir = ensure_dir(os.path.join(outdir, "masks"))
    met_dir  = ensure_dir(os.path.join(outdir, "metrics"))

    rows = []
    for ip in sorted(image_paths):
        res = model.predict(source=ip, imgsz=imgsz, conf=conf, verbose=False)[0]
        base = os.path.splitext(os.path.basename(ip))[0]

        cv2.imwrite(os.path.join(ann_dir, f"{base}_ann.jpg"), res.plot())
        n_inst, area_ratio = save_instance_masks(res, mask_dir, base)

        rows.append([os.path.basename(ip), n_inst, area_ratio])

    pd.DataFrame(rows, columns=["image","num_instances","spill_area_ratio"])\
      .to_csv(os.path.join(met_dir, "images_summary.csv"), index=False)

    print("Done. Check input folder.")

if __name__ == "__main__":
    main()


Done. Check input folder.
